In [ ]:
# Download dataset
!gdown 1aJbAliUb6kEmuQwMpw9YUTD_j8dCr352

Downloading...
From: https://drive.google.com/uc?id=1aJbAliUb6kEmuQwMpw9YUTD_j8dCr352
To: /content/chefkoch_rezepte.json
100% 17.2M/17.2M [00:00<00:00, 32.6MB/s]


In [ ]:
#%%
!pip install transformers

In [ ]:
from transformers import AutoTokenizer
import re
import json
from sklearn.model_selection import train_test_split

In [ ]:
with open('chefkoch_rezepte.json') as f:
    data = json.load(f)

In [ ]:
data[0]['Instructions']

'Die Eier hart kochen. Dann pellen und mit einem Eierschneider in Scheiben schneiden. Den Reis halbgar kochen und zur Seite stellen. Die Wurst (Kolbász) in dünne Scheiben schneiden.Den Knoblauch abziehen und fein würfeln. Die Zwiebel schälen, fein hacken und in etwas Fett glasig braten. Knoblauch und Hackfleisch dazu geben und so lange braten, bis das Hackfleisch schön krümelig wird. Den eigenen Saft nicht ganz verkochen lassen. Die Fleischmasse mit Salz, Pfeffer und Paprikapulver würzen.Das Sauerkraut kurz durchspülen, ausdrücken und abtropfen lassen (damit es nicht zu sauer wird). Das Sauerkraut in einen Topf geben und mit dem Kümmel und den Lorbeerblättern vermischen. Ca. 30 Minuten unter Zugabe von wenig Wasser bei niedriger Stufe dünsten.Eine feuerfeste Form mit etwas Öl einfetten und den Boden dünn mit Sauerkraut belegen. Darauf Kolbász und die Hälfte der in Scheiben geschnittene Eier verteilen, dann eine weitere dünne Schicht Sauerkraut drüber legen. Mit 1 Becher Schmand bedecke

In [ ]:
def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts['Instructions']).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  " # This makes no sense actually. Different recipes are just concatenated?!
    f.write(data)

train, test = train_test_split(data,test_size=0.15)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

#Train dataset length: 10361
#Test dataset length: 1829

Train dataset length: 10361
Test dataset length: 1829


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("anonymous-german-nlp/german-gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Test the baseline
prompt = "Die Eier"
input_ids = tokenizer.encode(prompt, return_tensors='pt').to('cuda')
output = model.generate(input_ids, max_length = 100, num_beams=12, no_repeat_ngram_size=2, early_stopping=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
tokenizer.decode(output[0], skip_special_tokens=True)

'Die Eier werden in einem Kühlschrank aufbewahrt.\nDas Badezimmer ist mit einer Dusche, einem Bidet und einer Badewanne mit Whirlpoolfunktion ausgestattet und bietet eine Hydromassage-Dusche, einen Haartrockner und Pflegeprodukte, die Sie gegen eine zusätzliche Gebühr nutzen können (Gebühr).\nDie Gäste können sich sowohl ihr Frühstück als auch Mittag- und Abendessen an einem Buffet zusammenstellen, das mit hausgemachtem Kuchen, Croissants, Aufschnitt, Käse, Wurst, Müsli,'

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("anonymous-german-nlp/german-gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2-gerchef", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()
trainer.save_model()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.848100
1000,2.180800


TrainOutput(global_step=1476, training_loss=2.3339627066924966, metrics={'train_runtime': 1568.3359, 'train_samples_per_second': 30.089, 'train_steps_per_second': 0.941, 'total_flos': 3082592747520000.0, 'train_loss': 2.3339627066924966, 'epoch': 3.0})

In [ ]:
model = AutoModelWithLMHead.from_pretrained('./gpt2-gerchef').to('cuda')

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1295: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
output = model.generate(input_ids, max_length = 100, num_beams=12, no_repeat_ngram_size=2, early_stopping=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
tokenizer.decode(output[0], skip_special_tokens=True)

'Die Eier pochieren. Dafür in einem mittleren Topf Essigwasser zum Kochen bringen und danach ein bisschen herunterdrehen, bis das Wasser simmert. Mit einem Schneebesen im Wasser einen Strudel erzeugen. In diesenstrudel die Eier schlagen. Wichtig ist, dass dasWasser an diesem Punkt noch in Bewegung ist. Danach den Deckel auf den Topf setzen und je nach Eiergröße 2 - 3 Minuten simmern lassen.Die Toastbrötchenhälften im Toaster, einer Pfanne'